### Path

In [1]:
import os
os.path

<module 'posixpath' from '/Users/iflab/Desktop/anaconda3/lib/python3.7/posixpath.py'>

/Users/iflab/Desktop/mix_platforms.csv

In [2]:
os.chdir(r'/Users/iflab/Desktop')

### Package

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from sklearn.model_selection import train_test_split

### Data

In [4]:
data = pd.read_csv('mix_platforms.csv', encoding='utf-8',low_memory=False)

In [5]:
data.drop(columns=['platforms','pledged_percent'],inplace=True)

In [6]:
data.set_index('project_id',inplace=True)

In [7]:
data['status']=data['status'].map({1:'success',0:'failed'})

In [8]:
data.head(3)

,status,category,location,backers,goal_usd,pledged_usd
project_id,,,,,,
620302213,success,Art,USD,6,0.01,100.00
9572984,failed,Film & Video,USD,0,0.15,0.00
1379346088,success,Art,MXN,7,0.49,16.41


In [ ]:
data['category'].loc[data['category']=='art']='Art'
data['category'].loc[data['category']=='comics']='Comics'
data['category'].loc[data['category']=='music']='Music'
data['category'].loc[data['category']=='design']='Design'
data['category'].loc[data['category']=='technology']='Technology'
data['category'].loc[data['category']=='food']='Food'
data['category'].loc[data['category']=='photography']='Photography'
data['category'].loc[data['category']=='theatre']='Theater'
data['category'].loc[data['category']=='fashion']='Fashion'
data['category'].loc[data['category']=='dance']='Dance'

In [9]:
data.shape

(593011, 6)

## No status

In [10]:
data.drop(columns=['status'],inplace=True)

## One hot

In [11]:
data=pd.get_dummies(data,drop_first=True)

In [12]:
data.head(3)

,backers,goal_usd,pledged_usd,category_Comics,category_Crafts,category_Dance,category_Design,category_Fashion,category_Film & Video,category_Food,...,location_EUR,location_GBP,location_HKD,location_JPY,location_MXN,location_NOK,location_NZD,location_SEK,location_SGD,location_USD
project_id,,,,,,,,,,,,,,,,,,,,,
620302213,6,0.01,100.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9572984,0,0.15,0.00,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1379346088,7,0.49,16.41,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [13]:
data.shape

(593011, 71)

## Train Test Spliting

In [14]:
data_v1 = data.copy()

In [15]:
X = data_v1.drop("pledged_usd", axis=1).values
y = data_v1["pledged_usd"].values.reshape(-1,1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
print(np.mean(y_train),np.std(y_train),'\n')
print(np.mean(y_test),np.std(y_test))

11818.4288975322 123761.02171133753 

12084.047838491548 119781.33735993227


## Model

### LightGBM

In [18]:
import lightgbm as lgb
gbm_model = lgb.LGBMRegressor()
gbm_model.fit(X_train,y_train)

/Users/iflab/Desktop/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
/Users/iflab/Desktop/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [19]:
train_scores = gbm_model.score(X_train,y_train)
print (train_scores)

0.7282709385017333


In [20]:
test_scores = gbm_model.score(X_test, y_test)
print (test_scores)

0.6637709636847648


In [21]:
train_gbm = train_scores
test_gbm = test_scores

### XGBoost

In [22]:
import xgboost as xgb
xgbc_model=xgb.XGBRegressor()
xgbc_model.fit(X_train,y_train)

[23:35:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [23]:
train_scores = xgbc_model.score(X_train,y_train)
print (train_scores)

0.8587786465034263


In [24]:
test_scores = xgbc_model.score(X_test, y_test)
print (test_scores)

0.7851935244434218


In [25]:
train_xgbc = train_scores
test_xgbc = test_scores

### Decision Tree Regressor

In [26]:
from sklearn.tree import DecisionTreeRegressor
tree_model=DecisionTreeRegressor(max_depth=4)
tree_model.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [27]:
train_scores = tree_model.score(X_train,y_train)
print (train_scores)

0.7889804642861942


In [28]:
test_scores = tree_model.score(X_test, y_test)
print (test_scores)

0.715237827566908


In [29]:
train_tree = train_scores
test_tree = test_scores

### Random Forest Regressor

In [30]:
from sklearn.ensemble import RandomForestRegressor
rf_model=RandomForestRegressor(max_depth=4)
rf_model.fit(X_train,y_train)

/Users/iflab/Desktop/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/iflab/Desktop/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [31]:
train_scores = rf_model.score(X_train,y_train)
print (train_scores)

0.7826958426189891


In [32]:
test_scores = rf_model.score(X_test, y_test)
print (test_scores)

0.72290469589181


In [33]:
train_rf = train_scores
test_rf = test_scores

### Lasso Regression

In [34]:
from sklearn.linear_model import Lasso,Ridge
lasso_model = Lasso()
lasso_model.fit(X_train,y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [35]:
train_scores = lasso_model.score(X_train,y_train)
print (train_scores)

0.5182253547519378


In [36]:
test_scores = lasso_model.score(X_test, y_test)
print (test_scores)

0.5942565281838805


In [37]:
train_ls = train_scores
test_ls = test_scores

### AdaBoost

In [38]:
from sklearn.ensemble import AdaBoostRegressor
ada_model = AdaBoostRegressor()
ada_model.fit(X_train,y_train)

/Users/iflab/Desktop/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=50, random_state=None)

In [39]:
train_scores = ada_model.score(X_train,y_train)
print (train_scores)

0.687292070707327


In [40]:
test_scores = ada_model.score(X_test, y_test)
print (test_scores)

0.5845346170212528


In [41]:
train_ada = train_scores
test_ada = test_scores

### SVM Regressor

### MLP Regressor

In [46]:
from sklearn.neural_network import MLPRegressor 
mlp_model=MLPRegressor() 
mlp_model.fit(X_train,y_train)

/Users/iflab/Desktop/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [47]:
train_scores = mlp_model.score(X_train,y_train) 
print (train_scores)

0.5246152668974065


In [48]:
test_scores = mlp_model.score(X_test, y_test) 
print (test_scores)

0.5980262445745379


In [49]:
train_mlp = train_scores
test_mlp = test_scores

### Ensemble

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor 
from sklearn.linear_model import Lasso,Ridge
from sklearn.svm import LinearSVR
import lightgbm as lgb
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import r2_score
import pickle

In [50]:
class Stacking:
    
    def __init__(self, x_train, y_train, x_test, y_test, ls_model_stack, model_final, type_label):
        self.trainx = x_train
        self.trainy = y_train
        self.testx = x_test
        self.testy = y_test
        self.ls_model_stack = ls_model_stack
        self.model_final = model_final
        self.type_label = type_label
        self.trainx_stacking_input = None
        self.trainy_stacking = None
        self.trainx_final = None
        self.trainy_final = None
        self.trainx_final_input = None
        self.testx_final_input = np.zeros((x_test.shape[0], len(ls_model_stack)))
    
    def spliting(self):
        if self.type_label == 'discrete':
            self.trainx_stacking_input, self.trainx_final, self.trainy_stacking, self.trainy_final = train_test_split (self.trainx, self.trainy, test_size= 0.3, random_state = 42, stratify = self.trainy)
        elif self.type_label == 'continuous':
            self.trainx_stacking_input, self.trainx_final, self.trainy_stacking, self.trainy_final = train_test_split (self.trainx, self.trainy, test_size= 0.3, random_state = 42)
        self.trainx_final_input = np.zeros((self.trainx_final.shape[0], len(self.ls_model_stack)))   
    
    def modeling_stack_training(self):
        for model, i in zip(self.ls_model_stack, range(len(self.ls_model_stack))):
            model.fit(self.trainx_stacking_input, self.trainy_stacking)
            output = model.predict(self.trainx_final)
            self.trainx_final_input[:, i] = output
            self.save_model(model, str(i))
            
    def modeling_final_training(self):
        self.model_final.fit(self.trainx_final_input, self.trainy_final)
        self.save_model(self.model_final, 'final')
        
    def predict_stack_testing(self):
        for model, i in zip(self.ls_model_stack, range(len(self.ls_model_stack))):
            output = model.predict(self.testx)
            self.testx_final_input[:, i] = output
    
    def predict_final_testing(self):
        self.testy_pred = self.model_final.predict(self.testx_final_input)
        self.testy_pred_prob=self.model_final.predict_proba(self.testx_final_input)
        return self.testy_pred, self.testy_pred_prob
    
    def scoring_testing(self):
        if self.type_label == 'discrete':
            fpr, tpr, thresholds = roc_curve(self.testy, self.testy_pred_prob[:, 1], pos_label=None)
            AUC=auc(fpr, tpr)
            return AUC
        elif self.type_label == 'continuous':
            r2 = r2_score(self.testy, self.testy_pred)
            return r2
    
    def save_model(self, model, filename):
        pickle.dump(model, open(filename, 'wb'))


In [51]:
model_stack=[xgb.XGBRegressor(),
             RandomForestRegressor(),
             DecisionTreeRegressor(),
             AdaBoostRegressor(),
             LinearSVR(),
             MLPRegressor(),
             lgb.LGBMRegressor()]
model_final = Lasso()
type_label = 'continuous'

In [52]:
stacking = Stacking(X_train, y_train, X_test, y_test, model_stack, model_final, type_label)
stacking.spliting()
stacking.modeling_stack_training()
stacking.modeling_final_training()
stacking.predict_stack_testing()
stacking.predict_final_testing()
stacking.scoring_testing()

[23:41:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/iflab/Desktop/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/iflab/Desktop/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/Users/iflab/Desktop/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/iflab/Desktop/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

[23:43:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.778163113595573

In [53]:
test_ensemble = ensemble.scoring_testing()

## Records

In [70]:
records = pd.DataFrame({
    'Model': ['LightGBM',
              'Lasso Regression', 
              'Random Forest Regressor',  
              'Adaptive Boosting (AdaBoost)',
              'Decision Tree Regressor',
              'Extreme Gradient Boosting (XGBoost)',
              'MLP',
              'SVM'],
    'R2_train': [train_gbm,
                 train_ls, 
                 train_rf,  
                 train_ada, 
                 train_tree, 
                 train_xgbc,
                 train_mlp,
                 train_svm],
    'R2_test': [test_gbm,
                test_ls, 
                test_rf,  
                test_ada, 
                test_tree, 
                test_xgbc,
                test_mlp,
                test_svm]})

records['R2_test']=round(records['R2_test'],4).values
records.sort_values(by='R2_test', ascending=False,inplace=True)
records.reset_index(drop=True,inplace=True)
records

,Model,R2_train,R2_test
0,Extreme Gradient Boosting (XGBoost),0.858779,0.7852
1,Random Forest Regressor,0.782696,0.7229
2,Decision Tree Regressor,0.788980,0.7152
3,LightGBM,0.728271,0.6638
4,MLP,0.524615,0.5980
5,Lasso Regression,0.518225,0.5943
6,Adaptive Boosting (AdaBoost),0.687292,0.5845
7,SVM,-15.773554,-19.5815


In [71]:
records = pd.DataFrame({
    'Model': [
              'LightGBM',
              'Lasso Regression', 
              'Random Forest Regressor',  
              'Adaptive Boosting (AdaBoost)',
              'Decision Tree Regressor',
              'Extreme Gradient Boosting (XGBoost)',
              'MLP',
              'Ensemble',
              'SVM'],
    'R2_test': [
                test_gbm,
                test_ls, 
                test_rf,  
                test_ada, 
                test_tree, 
                test_xgbc,
                test_mlp,
                test_ensemble,
                test_svm]})

records['R2_test']=round(records['R2_test'],4).values
records.sort_values(by='R2_test', ascending=False,inplace=True)
records.reset_index(drop=True,inplace=True)
records

,Model,R2_test
0,Extreme Gradient Boosting (XGBoost),0.7852
1,Ensemble,0.7782
2,Random Forest Regressor,0.7229
3,Decision Tree Regressor,0.7152
4,LightGBM,0.6638
5,MLP,0.5980
6,Lasso Regression,0.5943
7,Adaptive Boosting (AdaBoost),0.5845
8,SVM,-19.5815
